## 구글 드라이브 파일 다운로드

In [ ]:
!pip install gdown

In [ ]:
!gdown --id 14Wpnt6DPX93rtcH0RsvFZ3eFgtoA1K8K
!gdown --id 1AMiOTLuAqRgtZ624TfTHwW7fSSS5GbCh
!gdown --id 1CeRbGPXPm9RCsFe13eyoBe0N-wKw2DLg

## 파일 불러오기

In [ ]:
import pandas as pd

train_df = pd.read_csv('/kaggle/working/clean_train.csv')
test_df = pd.read_csv('/kaggle/working/clean_test.csv')

## 전처리

In [ ]:
# import re
# import string
# import os
# import nltk
# from nltk.corpus import stopwords
# from nltk.stem import PorterStemmer, WordNetLemmatizer
# import pandas as pd

# !unzip /root/nltk_data/corpora/wordnet.zip -d /root/nltk_data/corpora/

# nltk.download('stopwords')
# nltk.download('wordnet')

# stop_words = set(stopwords.words('english'))
# ps = PorterStemmer()
# lemmatizer = WordNetLemmatizer()

# def preprocess_text(text):
#     # 특수 문자 제거
#     text = re.sub(f"[{string.punctuation}]", " ", text)
    
#     # 소문자로 변환
#     text = text.lower()
    
#     # stopwords 제거
#     words = [word for word in text.split() if word not in stop_words]
    
#     # stemming
#     words = [ps.stem(word) for word in words]
    
#     # lemmatization
#     words = [lemmatizer.lemmatize(word) for word in words]
    
#     return " ".join(words)

# train_df["preprocessed_text"] = train_df["text"].apply(preprocess_text)
# test_df["preprocessed_text"] = test_df["text"].apply(preprocess_text)

# # Save preprocessed text to CSV files
# train_df.to_csv('clean_train.csv', index=False)
# test_df.to_csv('clean_test.csv', index=False)

# Dataset 클래스

In [ ]:
!pip install transformers

In [ ]:
from torch.utils.data import Dataset
from transformers import BertTokenizer

class NewsDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512, is_test=False):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.is_test = is_test
        self.encoded_dict = {}
        
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        if idx not in self.encoded_dict:
            text = self.dataframe.loc[idx, "preprocessed_text"]
            self.encoded_dict[idx] = self.tokenizer.encode_plus(
                text,
                add_special_tokens=True,
                max_length=self.max_length,
                padding='max_length',
                truncation=True,
                return_tensors='pt',
                return_attention_mask=True
            )
            
        item = {
            'input_ids': self.encoded_dict[idx]['input_ids'].squeeze(0),
            'attention_mask': self.encoded_dict[idx]['attention_mask'].squeeze(0),
            'input_text': self.dataframe.loc[idx, 'text'],  # Add the original text
            'index': idx  # Add the index
        }
        
        if not self.is_test:
            item['labels'] = self.dataframe.loc[idx, 'label']
            
        return item


## Dataset, DataLoader

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split

import random
import numpy as np

# Set random seeds for reproducibility
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_seed)

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')  # Changed to bert-base-uncased

# Create the datasets
train_data, val_data = train_test_split(train_df, test_size=0.2, stratify=train_df['label'], random_state=random_seed)

# Reset the index of the new dataframes
train_data = train_data.reset_index(drop=True)
val_data = val_data.reset_index(drop=True)

train_dataset = NewsDataset(train_data, tokenizer)
valid_dataset = NewsDataset(val_data, tokenizer)
test_dataset = NewsDataset(test_df, tokenizer, is_test=True)

# Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)  # Changed batch_size to 16
valid_dataloader = DataLoader(valid_dataset, batch_size=16, shuffle=False)  # Changed batch_size to 16
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)  # Changed batch_size to 16


## Model

In [ ]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=8)  # Changed to bert-base-uncased


## criterion, optimizer, metric

In [ ]:
from transformers import AdamW
from torch import nn
from sklearn.metrics import f1_score

# Criterion
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay=2e-2)

# Metric
def macro_f1_score(true_labels, predicted_labels):
    return f1_score(true_labels, predicted_labels, average='macro')


## Train loop

In [ ]:
import csv
import wandb
import tqdm

In [ ]:
import torch
import csv
from kaggle_secrets import UserSecretsClient
import io
import os

# Set random seeds
random_seed = 42
torch.manual_seed(random_seed)

# Wandb login
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_key")
wandb.login(key=secret_value_0)
os.environ["WANDB_SILENT"] = "true"
wandb.init(project='Dacon_GPT', name='bert-base-uncased_accumulation')

# Send model and hyperparameters to wandb
wandb.watch(model, log="all", log_freq=10)
wandb.config.epochs = 3
wandb.config.lr = 1e-5  # Changed lr value
wandb.config.weight_decay = 2e-2
wandb.config.batch_size = 16

# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define the number of gradient accumulation steps
gradient_accumulation_steps = 4  # Changed gradient_accumulation_steps to 4

In [ ]:
# Train and validation loop
for epoch in range(3):
    # Train loop
    model.train()
    train_loss = 0.0
    accumulation_steps = 0  # Initialize the accumulation step counter
    optimizer.zero_grad()  # Reset gradients tensors
    with tqdm.tqdm(train_dataloader, desc=f"Epoch {epoch+1}, Train", leave=False) as progress_bar:
        for batch in progress_bar:
            input_ids, attention_mask, labels = batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            loss = criterion(outputs.logits, labels)
            loss = loss / gradient_accumulation_steps  # Normalize our loss (if averaged)
            loss.backward()
            
            accumulation_steps += 1
            
            if accumulation_steps % gradient_accumulation_steps == 0: # Wait for several backward steps
                optimizer.step()                            # Now we can do an optimizer step
                optimizer.zero_grad()                       # Reset gradients tensors
                accumulation_steps = 0
            
            train_loss += loss.item()

            progress_bar.set_postfix({'loss': train_loss / ((len(train_dataset) * gradient_accumulation_steps) // wandb.config.batch_size)})

    wandb.log({'train_loss': train_loss / (len(train_dataloader) // gradient_accumulation_steps)})

    # Validation loop
    model.eval()
    valid_loss = 0.0
    validation_results = []  # Initialize an empty list to store validation results
    with torch.no_grad(), tqdm.tqdm(valid_dataloader, desc=f"Epoch {epoch+1}, Valid", leave=False) as progress_bar:
        for batch in progress_bar:
            input_ids, attention_mask, labels = batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['labels'].to(device)
            input_text, index = batch['input_text'], batch['index']
            outputs = model(input_ids, attention_mask=attention_mask)
            loss = criterion(outputs.logits, labels)
            valid_loss += loss.item()

            progress_bar.set_postfix({'loss': valid_loss / (len(valid_dataloader))})
            
            true_labels = labels.cpu().numpy()
            predicted_labels = torch.argmax(outputs.logits, dim=1).cpu().numpy()
            
            # Store the validation results (text, index, predicted_labels, true_labels)
            for text, idx, pred_label, true_label in zip(input_text, index, predicted_labels, true_labels):
                validation_results.append((text, idx, pred_label, true_label))

    wandb.log({'valid_loss': valid_loss / len(valid_dataloader)})

    valid_f1 = macro_f1_score([result[3] for result in validation_results], [result[2] for result in validation_results])
    wandb.log({'valid_f1': valid_f1})
    
    print(f"Epoch {epoch + 1}, Train Loss: {train_loss / (len(train_dataloader) // gradient_accumulation_steps)}, Valid Loss: {valid_loss / len(valid_dataloader)}, Valid Macro F1: {valid_f1}")


### validation.csv

In [ ]:
# Save the validation results to a CSV file
with open('validation_results.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['text', 'index', 'predicted_label', 'true_label'])
    for result in validation_results:
        csv_writer.writerow(result)


## 추론

In [ ]:
# Test loop
model.eval()
test_results = []  # Initialize an empty list to store test results
with torch.no_grad():
    for batch in tqdm.tqdm(test_dataloader, desc="Test", leave=False):
        input_ids, attention_mask = batch['input_ids'].to(device), batch['attention_mask'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        predicted_labels = torch.argmax(outputs.logits, dim=1).cpu().tolist()
        test_results.extend(predicted_labels)

# Load the sample submission file
submission_df = pd.read_csv('/kaggle/working/sample_submission.csv')

# Overwrite the label column with the test_results
submission_df['label'] = test_results

# Save the updated submission file to a CSV file
submission_df.to_csv('submission.csv', index=False)
